## Synthetic data

In [1]:
%load_ext autoreload 
%autoreload 2

import sys
import ast

import pandas as pd
import numpy as np

# IT'S 2025 WHY HASN'T SOMEONE FIXED THIS
if ".." not in sys.path:
    sys.path.insert(0, "..")
from src import aposteriori

import tasks.run_helper

In [2]:
syn_df = pd.read_csv(
    "../data/synthetic.csv",
    converters={
        "annot_personality_characteristics": ast.literal_eval,
        "Toxicity": ast.literal_eval,
        "Argument Quality": ast.literal_eval,
        "age_annot": ast.literal_eval,
        "sex_annot": ast.literal_eval,
        "sexual_orientation_annot": ast.literal_eval,
        "current_employment_annot": ast.literal_eval,
        "education_level_annot": ast.literal_eval,
    },
)
syn_df["comment_key"] = syn_df.message + syn_df.conv_id
syn_df["fake_index"] = 1

syn_df.Toxicity = syn_df.Toxicity.apply(lambda x: [int(tox) for tox in x])

syn_df.age_annot = syn_df.age_annot.apply(
    lambda ls: [int(x) for x in ls]
).apply(lambda x: pd.cut(x, bins=4))

In [3]:
tasks.run_helper.results(
    syn_df,
    discussion_id_col="fake_index",
    sdb_column="age_annot",
    value_col="Toxicity",
    comment_key_col="comment_key",
)

1          
                   value    pvalue
(20.956, 32.0]  0.007667  0.655581
(54.0, 65.0]   -0.040672  0.643743
(32.0, 43.0]    0.048232  0.640643
(43.0, 54.0]    0.012311  0.647125

In [4]:
tasks.run_helper.results(
    syn_df,
    discussion_id_col="fake_index",
    sdb_column="sexual_orientation_annot",
    value_col="Toxicity",
    comment_key_col="comment_key",
)

1          
                 value    pvalue
Lesbian       0.000000  1.000000
Pansexual     0.000000  1.000000
Homosexual    0.000000  1.000000
Heterosexual -0.019413  0.672773
Asexual       0.000000  1.000000
Bisexual      0.000000  1.000000

In [5]:
tasks.run_helper.results(
    syn_df,
    discussion_id_col="fake_index",
    sdb_column="education_level_annot",
    value_col="Toxicity",
    comment_key_col="comment_key",
)

1          
                        value    pvalue
Master's            -0.015614  0.657272
Some College         0.048960  0.642052
Bachelor's           0.018097  0.646280
PhD                 -0.041467  0.653326
No formal education  0.000000  1.000000

In [6]:
tasks.run_helper.results(
    syn_df,
    discussion_id_col="fake_index",
    sdb_column="current_employment_annot",
    value_col="Toxicity",
    comment_key_col="comment_key",
)

1       
                     value pvalue
Poet                   0.0    1.0
Game Developer         0.0    1.0
Research Scientist     0.0    1.0
Retired Philosopher    0.0    1.0
Stock Trader           0.0    1.0
Historian              0.0    1.0
Registered Nurse       0.0    1.0
Travel Blogger         0.0    1.0
Farmer                 0.0    1.0
Botanist               0.0    1.0
Cybersecurity Expert   0.0    1.0

In [7]:
tasks.run_helper.results(
    syn_df,
    discussion_id_col="fake_index",
    sdb_column="sex_annot",
    value_col="Toxicity",
    comment_key_col="comment_key",
)

1          
               value    pvalue
female      0.033293  0.647407
male       -0.000652  0.664036
non-binary  0.000000  1.000000

In [8]:
syn_df["random"] = syn_df.sex_annot.apply(
    lambda x: [np.random.randint(1, 10) for _ in range(len(x))]
)
tasks.run_helper.results(
    syn_df,
    discussion_id_col="fake_index",
    sdb_column="random",
    value_col="Toxicity",
    comment_key_col="comment_key",
)

1          
      value    pvalue
7 -0.021579  0.829689
6  0.078753  0.819235
2 -0.011570  0.829996
8 -0.015002  0.826189
4  0.018416  0.828185
1  0.006258  0.834752
3  0.060209  0.824854
5  0.034684  0.811415
9  0.044946  0.810419